# Ellipticity vs Pointing error
Craig Lage - 14-Jan-26

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.utils import dayObsIntToString

In [ ]:
dayObs = 20260113
imageTypes = ['acq', 'science', 'cwfs', 'engtest']
imageTable = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
imageTable = imageTable.sort_index()

In [ ]:
for col in imageTable.columns:
    print(col)

In [ ]:
#imageTable = imageTable[imageTable['Image type'] == 'science']
ptg_errors = []
es = []
for i in range(len(imageTable)):
    if i < 50:
        continue
    try:
        dRA = imageTable['delta Ra (arcsec)'].iloc[i]
        dDec = imageTable['delta Dec (arcsec)'].iloc[i]
        el = imageTable['Elevation'].iloc[i]
        ptg_error = np.sqrt(dDec**2 + (dRA * np.cos(np.pi * el / 180.0))**2)
        e1 = imageTable['PSF e1'].iloc[i]
        e2 = imageTable['PSF e2'].iloc[i]
        e = np.sqrt(e1**2 + e2**2)
        if np.isnan(e) or np.isnan(ptg_error):
            continue
        es.append(e)
        ptg_errors.append(ptg_error)
    except:
        continue
print(len(es), len(ptg_errors))

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(10,10))
ax.scatter(ptg_errors, es)
fit = np.polyfit(ptg_errors, es, 1)
xs = np.linspace(0, 25, 100)
ys = fit[1] + fit[0] * xs
ax.plot(xs, ys, ls='--', color='red')
ax.set_title(f"Ellipticity vs Pointing error {dayObs}", fontsize=18)
ax.set_xlabel("Total pointing error (arcsec)")
ax.set_ylabel("Ellipticity e")
fig.savefig(f"/home/cslage/DATA/Ellipticity_vs_Pointing_Error{dayObs}.png")